This notebook performs the synthesis using our proposed synthesization approach for the training data only (excludes the holdout data). The synthesis model is a CART.

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
from sklearn.mixture import GaussianMixture
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn import preprocessing

from scipy.stats import ks_2samp

from sklearn.tree import DecisionTreeRegressor

import itertools

from sklearn.neighbors import KernelDensity

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

from bayesian_bootstrap import bayesian_bootstrap

rng = np.random.RandomState(42)

Steps for CART estimation of pmse ratio.

* calculate the pMSE between pairs of synthetic data sets generated from the same original data
* the pairs can be used to estimate the expected pMSE even when the synthesizing model is incorrect since both data are drawn from the same distribution
* for most large complex data sets, synthesized by CART models, the expected pMSE from pairs will be close to, or slightly lower than the null pMSE

In [ ]:
def pmse_ratio(original_data, synthetic_data):
    
    N_synth = synthetic_data.shape[0]
    N_orig = original_data.shape[0]
    
    # combine original and synthetic datasets
    full_X = pd.concat([original_data, synthetic_data], axis=0).reset_index(drop=True)
    
    # generate interactions and powers of variables
    poly = PolynomialFeatures(3, interaction_only=False, include_bias=False)
    
    full_X = poly.fit_transform(full_X)

    # scale the combined dataset
    full_X = preprocessing.StandardScaler().fit_transform(full_X)
    
    c = N_synth/(N_synth+N_orig)

    y = np.repeat([0, 1], repeats=[N_orig, N_synth])
    
    pMSE_model = LogisticRegression(penalty=None, max_iter=1000).fit(full_X, y)
    
    probs = pMSE_model.predict_proba(full_X)
    
    pMSE = 1/(N_synth+N_orig) * np.sum((probs[:,1] - c)**2)
    
    e_pMSE = 2*(full_X.shape[1])*(1-c)**2 * c/(N_synth+N_orig)
        
    return pMSE/e_pMSE

Import the data.

In [ ]:
# import standardized lat/long location data
train_data = pd.read_csv("Data/cleaned_ipums_data.csv")

***

In [ ]:
train_data

In [ ]:
np.unique(train_data.non_white)

In [ ]:
np.unique(train_data.SEX)

Let's start the synthesis with sampling from the joint distribution of `non_white` and `SEX`. Then use CART to synthesize the three continuous variables.

Write a function to estimate the joint distribution and sample from it. Will be used in the optimizer.

In [ ]:
def categorical_sample(training_data):
    
    # probability for each combination of values
    pvals = (pd.crosstab(train_data.non_white, train_data.SEX)/train_data.shape[0]).values.flatten()
    
    # sample from multinomial distribution - how many occurrences of each 'class'
    new_counts = rng.multinomial(n=train_data.shape[0], pvals=pvals)
    
    # create a data frame with the new samples
    new_data = pd.DataFrame(np.vstack([np.repeat([['0', '0']], new_counts[0], axis=0),
           np.repeat([['0', '1']], new_counts[1], axis=0),
           np.repeat([['1', '0']], new_counts[2], axis=0),
           np.repeat([['1', '1']], new_counts[3], axis=0)]))
    
    cols = new_data.columns.values
    
    new_data[cols] = new_data[cols].apply(pd.to_numeric)
    
    return new_data

Test the function.

***

# Full Sequential Synthesis Driven by Bayesian Optimization

Function to be used in Bayesian bootstrap.

In [ ]:
def stat(x):
    return x

Function for training a CART model on a continuous variable. Should be the same as for a categorical variable but with the additional step of estimating a kernel density and sampling new values from that density.

***

Test the kernel density estimate and sampling.

In [ ]:
# list for synthetic datasets
sXs = []
    
# generate and store number_synthetic_datasets synthetic datasets
for i in range(3):
    sX = categorical_sample(train_data)
    sX.columns = ['non_white', 'SEX']
    sXs.append(sX)

In [ ]:
current_synthetic_datasets = sXs
number_synthetic_datasets = 3 
mb=5
covariate_array = ['non_white', 'SEX']
target = 'INCWAGE'

In [ ]:
cart = DecisionTreeRegressor(min_samples_leaf=mb, random_state=rng)

In [ ]:
cart.fit(X=train_data.loc[:, covariate_array], y=train_data.loc[:, target])

In [ ]:
node_indicators = cart.decision_path(train_data.loc[:, covariate_array]).toarray()

In [ ]:
node_indicators

In [ ]:
node_outcomes = [train_data[target][node_indicators[:,x] == 1] for x in np.arange(node_indicators.shape[1])]

In [ ]:
i = 1

In [ ]:
# resample values according to a Bayesian bootstrap
bst_vals = [bayesian_bootstrap(X=np.array(x), 
                               statistic=stat,
                               n_replications=1,
                               resample_size=len(x))[0] for x in node_outcomes]

In [ ]:
# compute which leaf each sensitive data record ends up in
synth_leaves = cart.apply(current_synthetic_datasets[i].loc[:, covariate_array])

In [ ]:
synth_leaves

In [ ]:
new_var = np.zeros(len(synth_leaves))

In [ ]:
new_var

In [ ]:
for i in np.arange(node_indicators.shape[1]):
    

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth="scott").fit(bst_vals[2].reshape(-1,1))

In [ ]:
log_dens = kde.score_samples(np.linspace(-2, 4, 100)[:, np.newaxis])

In [ ]:
plt.plot(np.linspace(-2, 4, 100)[:, np.newaxis], np.exp(log_dens))

Now generate some samples.

In [ ]:
sample_vals = kde.sample(np.sum(synth_leaves==2)).flatten()
to_replace = np.any([sample_vals < np.min(bst_vals[2]), sample_vals > np.max(bst_vals[2])], axis=0)
while np.sum(to_replace) > 0:
    sample_vals[to_replace] = kde.sample(np.sum(to_replace)).flatten()
    to_replace = np.any([sample_vals < np.min(bst_vals[2]), sample_vals > np.max(bst_vals[2])], axis=0)

In [ ]:
for j, x in enumerate(np.arange(node_indicators.shape[1])):
    new_var[synth_leaves==x] = KernelDensity(kernel='gaussian', bandwidth="scott").fit(bst_vals[x].reshape(-1,1)).sample(np.sum(synth_leaves==x)).flatten()

***

In [ ]:
def continuous_cart_synthesis(current_synthetic_datasets, train_data, number_synthetic_datasets, mb, covariate_array, target):
        
    cart = DecisionTreeRegressor(min_samples_leaf=mb, random_state=rng)
    
    cart.fit(X=train_data.loc[:, covariate_array], y=train_data.loc[:, target])
    
    node_indicators = cart.decision_path(train_data.loc[:, covariate_array]).toarray()
    
    node_outcomes = [train_data[target][node_indicators[:,x] == 1] for x in np.arange(node_indicators.shape[1])]
    
    for i in range(number_synthetic_datasets):
        
        # resample values according to a Bayesian bootstrap
        bst_vals = [bayesian_bootstrap(X=np.array(x), 
                                       statistic=stat,
                                       n_replications=1,
                                       resample_size=len(x))[0] for x in node_outcomes]
    
        # compute which leaf each synthetic data record ends up in
        synth_leaves = cart.apply(current_synthetic_datasets[i].loc[:, covariate_array])
    
        new_var = np.zeros(len(synth_leaves))
    
        for j, x in enumerate(np.arange(node_indicators.shape[1])):
            
            kde = KernelDensity(kernel='gaussian', bandwidth="scott").fit(bst_vals[x].reshape(-1,1))
            
            sample_vals = kde.sample(np.sum(synth_leaves==x)).flatten()
            to_replace = np.any([sample_vals < np.min(bst_vals[x]), sample_vals > np.max(bst_vals[x])], axis=0)
            while np.sum(to_replace) > 0:
                sample_vals[to_replace] = kde.sample(np.sum(to_replace)).flatten()
                to_replace = np.any([sample_vals < np.min(bst_vals[x]), sample_vals > np.max(bst_vals[x])], axis=0)
            
            new_var[synth_leaves==x] = sample_vals
        
        new_var = pd.Series(new_var)
        
        new_var.name = target
        
        current_synthetic_datasets[i] = pd.concat([current_synthetic_datasets[i], new_var], axis=1)
        
    return current_synthetic_datasets

***

Write function to train all models and generate the synthetic dataset, then evaluate the pMSE ratio.

In [ ]:
def train_models(#overall parameters
                 train_data,
                 number_synthetic_datasets,
                 # hyperparameters for GMM, end with underscore means Bayesian optimization will choose
                 # number_gmm_initializations,
                 # num_components_,
                 # hyperparameters for CART, end with underscore means Bayesian optimization will choose
                 mb_INCWAGE_,
                 mb_educ_,
                 mb_exp_):
    
    num_samples = train_data.shape[0]
    
    ########## Code for GMM ############
    
    # fit GMM model
    # GMM = GaussianMixture(num_components_, n_init=number_gmm_initializations, init_params="k-means++", random_state=rng).fit(train_data.loc[:,["latitude", "longitude"]])
    
    # list for synthetic datasets
    sXs = []
    
    # generate and store number_synthetic_datasets synthetic datasets
    for i in range(number_synthetic_datasets):
        sX = categorical_sample(train_data)
        sX.columns = ['non_white', 'SEX']
        sXs.append(sX)
        
    ####################################################################################################
        
    ########### Code for INCWAGE ##########
    
    sXs = continuous_cart_synthesis(current_synthetic_datasets=sXs,
                                    train_data=train_data,
                                    number_synthetic_datasets=number_synthetic_datasets,
                                    mb=mb_INCWAGE_,
                                    covariate_array=['non_white', 'SEX'],
                                    target="INCWAGE")
        
    ####################################################################################################
        
    ########### Code for years_of_educ CART ##########
    sXs = continuous_cart_synthesis(current_synthetic_datasets=sXs,
                                    train_data=train_data,
                                    number_synthetic_datasets=number_synthetic_datasets,
                                    mb=mb_educ_,
                                    covariate_array=['non_white', 'SEX', 'INCWAGE'],
                                    target="years_of_educ")
        
    ####################################################################################################
    
    ########### Code for potential_experience CART ##########
    
    sXs = continuous_cart_synthesis(current_synthetic_datasets=sXs,
                                    train_data=train_data,
                                    number_synthetic_datasets=number_synthetic_datasets,
                                    mb=mb_exp_,
                                    covariate_array=['non_white', 'SEX', 'INCWAGE', 'years_of_educ'],
                                    target="potential_experience")
    
    sXs = [x.loc[:,list(train_data.columns.values)] for x in sXs]
    
    ###### Calculate ks distances ######
    pmse_ratios = [pmse_ratio(train_data, Y) for Y in sXs]
    
    return pmse_ratios, sXs

In [ ]:
def optimize_models(train_data,
                    number_synthetic_datasets,
                    # number_gmm_initializations,
                    random_state):

    def evaluate_models(mb_INCWAGE_, mb_educ_, mb_exp_):

        pmse_ratios, _ = train_models(train_data=train_data,
                                      number_synthetic_datasets=number_synthetic_datasets,
                                      # number_gmm_initializations=number_gmm_initializations,
                                      # num_components_=int(num_components_),
                                      mb_INCWAGE_=int(mb_INCWAGE_),
                                      mb_educ_=int(mb_educ_),
                                      mb_exp_=int(mb_exp_))

        return -1 * ((1 - np.mean(pmse_ratios))**2)

    optimizer = BayesianOptimization(
        f=evaluate_models,
        pbounds={
            # "num_components_": (200, 800.99),
            "mb_INCWAGE_": (3, 300.99),
            "mb_educ_": (3, 300.99),
            "mb_exp_": (3, 300.99)
        },
        random_state=random_state)

    utility = UtilityFunction(kind="ei", xi=1e-02)
    optimizer.maximize(init_points=5, n_iter=10, acquisition_function=utility)
    print("Final Result: ", optimizer.max)
    return optimizer.max, optimizer

The default value of $\alpha = 1e-06$.

In [ ]:
nsd = 5
# ngi = 5
# random_states = [np.random.RandomState(1234), np.random.RandomState(4321), np.random.RandomState(10620), np.random.RandomState(91695), np.random.RandomState(31296)]
random_states = [np.random.RandomState(1234)]

In [ ]:
optimization_results = [optimize_models(train_data=train_data, number_synthetic_datasets=nsd, random_state=r) for r in random_states]

***

In [ ]:
run_targets = [np.minimum.accumulate(-i[1].space.target) for i in optimization_results]

In [ ]:
plt.plot(run_targets[0])
plt.scatter(np.arange(len(run_targets[0])), run_targets[0], s=6)
plt.plot(run_targets[1])
plt.scatter(np.arange(len(run_targets[1])), run_targets[1], s=6)
plt.plot(run_targets[2])
plt.scatter(np.arange(len(run_targets[2])), run_targets[2], s=6)
plt.plot(run_targets[3])
plt.scatter(np.arange(len(run_targets[3])), run_targets[3], s=6)
plt.plot(run_targets[4])
plt.scatter(np.arange(len(run_targets[4])), run_targets[4], s=6)
plt.title("Running Minimum Objective Value for CART Synthesis")
plt.ylim(-0.01, 0.47)
plt.show()

Choose the params that gave the best objective value across all random seeds.

In [ ]:
best_params = optimization_results[np.argmax([x[0]['target'] for x in optimization_results])][0]

In [ ]:
best_params

***

Generate 1000 synthetic datasets, choose the 20 that have the pMSE closest to 1.

In [ ]:
pmse_ratios, full_sXs = train_models(train_data=train_data,
                                                                          number_synthetic_datasets=nsd,
                                                                          # hyperparameters for GMM
                                                                          # number_gmm_initializations=ngi,
                                                                          # num_components_=int(best_params['params']['num_components_']),
                                                                          # hyperparameters for CART, end with underscore means Bayesian optimization will choose
                                                                          mb_INCWAGE_=int(best_params['params']['mb_INCWAGE_']),
                                                                          mb_educ_=int(best_params['params']['mb_educ_']),
                                                                          mb_exp_=int(best_params['params']['mb_exp_']))

In [ ]:
np.mean(pmse_ratios)

In [ ]:
plt.violinplot(pmse_ratios)
plt.xlabel("Density")
plt.ylabel("pMSE Ratio")
plt.title("Distribution of pMSE Ratios")
plt.show()

In [ ]:
temp = full_sXs[0]

In [ ]:
temp

In [ ]:
train_data

In [ ]:
np.mean(temp.non_white)

In [ ]:
np.mean(train_data.non_white)

In [ ]:
pd.crosstab(train_data.non_white, train_data.SEX)/train_data.shape[0]

In [ ]:
pd.crosstab(temp.non_white, temp.SEX)/temp.shape[0]

In [ ]:
np.mean(temp.potential_experience)

In [ ]:
np.mean(train_data.potential_experience)

***

In [ ]:
synthetic_data = temp
original_data = train_data

In [ ]:
N_synth = synthetic_data.shape[0]
N_orig = original_data.shape[0]
    
# combine original and synthetic datasets
full_X = pd.concat([original_data, synthetic_data], axis=0).reset_index(drop=True)
    
# generate interactions and powers of variables
poly = PolynomialFeatures(1, interaction_only=False, include_bias=False)
    
full_X = poly.fit_transform(full_X)

# scale the combined dataset
full_X = preprocessing.StandardScaler().fit_transform(full_X)
    
c = N_synth/(N_synth+N_orig)

y = np.repeat([0, 1], repeats=[N_orig, N_synth])
    
pMSE_model = LogisticRegression(penalty=None, max_iter=1000).fit(full_X, y)
    
probs = pMSE_model.predict_proba(full_X)
    
pMSE = 1/(N_synth+N_orig) * np.sum((probs[:,1] - c)**2)
    
e_pMSE = 2*(full_X.shape[1])*(1-c)**2 * c/(N_synth+N_orig)

In [ ]:
pMSE/e_pMSE

In [ ]:
pMSE_model.coef_

In [ ]:
full_X

In [ ]:
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant

In [ ]:
full_X_sm = add_constant(full_X)

In [ ]:
full_X_sm

In [ ]:
lm = Logit(endog = y,
           exog = full_X_sm)

In [ ]:
log_model = lm.fit()

In [ ]:
log_model.summary()

***

# Save the synthetic datasets.

In [ ]:
for i, sX in enumerate(full_sXs):
    sX.to_csv("Data/synthetic_datasets/cart_mb_logistic_pmse_" + str(i) + ".csv", index=False)

***